In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
path = '/content/drive/My Drive/A3/'

In [0]:
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from mpl_toolkits.axes_grid1 import ImageGrid
import random
from itertools import cycle
from sklearn.externals import joblib

In [0]:
# train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(path, train=True, download=True,
#                                            transform=torchvision.transforms.Compose([
#                                                      torchvision.transforms.ToTensor(),
#                                                      torchvision.transforms.Normalize((0.1307,), (0.3081,))
#                                                      ])), batch_size=64, shuffle=True)

In [0]:
# test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST(path, train=False, download=True,
#                                            transform=torchvision.transforms.Compose([
#                                                      torchvision.transforms.ToTensor(),
#                                                      torchvision.transforms.Normalize((0.1307,), (0.3081,))
#                                                      ])), batch_size=64, shuffle=True)

In [86]:
encoder = Encoder()
encoder = encoder.cuda()
decoder = Decoder()
decode = decoder.cuda()
adv = Discriminator()
adv = adv.cuda()

train_loader = MNIST_Paired()

encoder.apply(weights_init)
decoder.apply(weights_init)

z = torch.FloatTensor(64, 16)
z = z.cuda()

adversarial_loss = torch.nn.BCELoss()
adversarial_loss.cuda()

auto_encoder_optimizer = optim.Adam(list(encoder.parameters())+list(decoder.parameters()), lr=1e-3, betas=(0.9, 0.999))
adversary_optimizer = optim.Adam(list(adv.parameters()), lr=2e-3, betas=(0.5, 0.9))

auto_encoder_scheduler = optim.lr_scheduler.StepLR(auto_encoder_optimizer, step_size=20, gamma=0.1)
adversary_scheduler = optim.lr_scheduler.StepLR(adversary_optimizer, step_size=20, gamma=0.1)

loader = cycle(torch.utils.data.DataLoader(train_loader, batch_size=64, shuffle=True, num_workers=0, drop_last=True))

for epoch in range(100):
    loss = 0
    loss_plot = []
    auto_encoder_scheduler.step()
    adversary_scheduler.step()
    tempList = []
    for iteration in range(int(len(train_loader)/64)):
        #AE training
        auto_encoder_optimizer.zero_grad()
        
        #Sample a triplet of samples with two data points having same label, third one can be random
        x_1, x_2, labels = next(loader)
        x_3, _, id_3 = next(loader)
        #Convert to cuda
        x_1 = x_1.cuda()
        x_2 = x_2.cuda()
        x_3 = x_3.cuda()
        id_3 = id_3.cuda()
        #Computing the codes by passing through encoder
        z_mean_1, z_var_1, s_1 = encoder(Variable(x_1))
        z_mean_2, z_var_2, s_2 = encoder(Variable(x_2))
        z_mean_3, z_var_3, s_3 = encoder(Variable(x_3))
        #Sampling style_latent_variables using reparameterisation trick
        z_1 = reparameterize(True, z_mean_1, z_var_1)
        z_2 = reparameterize(True, z_mean_2, z_var_2)
        z_3 = reparameterize(True, z_mean_3, z_var_3)
        #Reconstructing the image by passing through the decoder
        reconstructed_X11 = decoder(z_1, s_1)
        reconstructed_X12 = decoder(z_1, s_2)
        #Computing the reconstruction loss and updating the gradient
        reconstruction_loss_11 = mse_loss(reconstructed_X11, Variable(x_1))
        reconstruction_loss_12 = mse_loss(reconstructed_X12, Variable(x_1))
        reconstruction_loss = (reconstruction_loss_11 + reconstruction_loss_12)/2. 
        #loss += reconstruction_loss.data.storage().tolist()[0]
        reconstruction_loss.backward(retain_graph=True)
        #Compute the generation on random data point i.e., x_3 and update the gradients based on adversarial loss
        reconstructed_X13 = decoder(z_1, s_3)
        adv_loss_1 = (-0.5*torch.sum(1+z_var_1-z_mean_1.pow(2)-z_var_1.exp()))/(64*1*28*28)
        z.normal_(0.,1.)
        reconstructed_X_3 = decoder(z, s_3)
        adv_loss_2 = (-0.5*torch.sum(1+torch.var(z)-torch.mean(z).pow(2)-torch.var(z).exp()))/(64*1*28*28)
        adv_loss = (adv_loss_1+adv_loss_2)/2.
        adv_loss.backward(retain_graph=True)
        auto_encoder_optimizer.step()
        #Adversary training
        adversary_optimizer.zero_grad()
        x_1, _, id_1 = next(loader)
        x_2, _, id_2 = next(loader)
        
        x_1 = x_1.cuda()
        x_2 = x_2.cuda()
        id_1 = id_1.cuda()
        id_2 = id_2.cuda()
        
        z_mean_1, z_var_1, s_1 = encoder(Variable(x_1))
        z_mean_2, z_var_2, s_2 = encoder(Variable(x_2))
        
        z_1 = reparameterize(True, z_mean_1, z_var_1)
        z_2 = reparameterize(True, z_mean_2, z_var_2)
        
        reconstructed_X11 = decoder(z_1, s_1)
        reconstructed_X12 = decoder(z_1, s_2)
        
        adv_loss = (-0.5*torch.sum(1+z_var_1-z_mean_1.pow(2)-z_var_1.exp()))/(64*1*28*28)
        
        adv_loss.backward(retain_graph=True)
        adversary_optimizer.step()
        
        cnt = 0
        if (iteration+1) % 100 == 0:
            cnt += 1
            print('Epoch\t'+str(epoch),'Iter\t'+str(iteration), 
                  'Loss\t' + str(reconstruction_loss.data.storage().tolist()[0]))
            loss += reconstruction_loss.data.storage().tolist()[0]
            loss_plot.append(loss/cnt)
        for i in range(len(z_1.cpu().data.numpy().tolist())):
                temp = []
                temp.append(z_1.cpu().data.numpy().tolist()[i])
                temp.append(labels[i].cpu().data.numpy().tolist())
                tempList.append(temp) 
   
    if (epoch+1) % 20 == 0 or (epoch + 1) == 100:
        temp = []
        torch.save(encoder.state_dict(), path + 'encoder.weights')
        torch.save(decoder.state_dict(), path + 'decoder.weights')


        inputs_1, inputs_2, __ = next(loader)
        inputs_3, _, __ = next(loader)
        inputs_1, inputs_2, inputs_3 = inputs_1.cuda(), inputs_2.cuda(), inputs_3.cuda()
        z_mean_1, z_var_1, s_1 = encoder(Variable(inputs_1))
        z_mean_2, z_var_2, s_2 = encoder(Variable(inputs_2))
        z_mean_3, z_var_3, s_3 = encoder(Variable(inputs_3))

        z_1 = reparameterize(False, z_mean_1, z_var_1)
        z_3 = reparameterize(False, z_mean_3, z_var_3)
        
        reconstructed_X_12 = decoder(z_1, s_2)
        reconstructed_X_32 = decoder(z_3, s_2)

        image_batch = np.transpose(inputs_1.cpu().numpy(), (0, 2, 3, 1))
        image_batch = np.concatenate((image_batch, image_batch, image_batch), axis=3)
        imshow_grid(image_batch, name=str(epoch) + '_original', save=True)

#         reconstructed_x = np.transpose(reconstructed_X_12.cpu().data.numpy(), (0, 2, 3, 1))
#         reconstructed_x = np.concatenate((reconstructed_x, reconstructed_x, reconstructed_x), axis=3)
#         imshow_grid(reconstructed_x, name=str(epoch) + '_target', save=True)

        style_batch = np.transpose(inputs_3.cpu().numpy(), (0, 2, 3, 1))
        style_batch = np.concatenate((style_batch, style_batch, style_batch), axis=3)
        imshow_grid(style_batch, name=str(epoch) + '_style', save=True)

        reconstructed_style = np.transpose(reconstructed_X_32.cpu().data.numpy(), (0, 2, 3, 1))
        reconstructed_style = np.concatenate((reconstructed_style, reconstructed_style, reconstructed_style), axis=3)
        imshow_grid(reconstructed_style, name=str(epoch) + '_style_target', save=True)
        
        
        joblib.dump(tempList, path+'tsne.sav')

Epoch	0 Iter	99 Loss	0.6503085494041443
Epoch	0 Iter	199 Loss	0.5767725706100464
Epoch	0 Iter	299 Loss	0.5525031685829163
Epoch	0 Iter	399 Loss	0.5341758728027344
Epoch	0 Iter	499 Loss	0.5366477370262146
Epoch	0 Iter	599 Loss	0.5277217030525208
Epoch	0 Iter	699 Loss	0.5190153121948242
Epoch	0 Iter	799 Loss	0.5105174779891968
Epoch	0 Iter	899 Loss	0.5038754343986511
Epoch	1 Iter	99 Loss	0.5094743967056274
Epoch	1 Iter	199 Loss	0.5086038112640381
Epoch	1 Iter	299 Loss	0.5139362812042236
Epoch	1 Iter	399 Loss	0.5065673589706421
Epoch	1 Iter	499 Loss	0.5139414072036743
Epoch	1 Iter	599 Loss	0.513563871383667
Epoch	1 Iter	699 Loss	0.5038045048713684
Epoch	1 Iter	799 Loss	0.5005918741226196
Epoch	1 Iter	899 Loss	0.4925180673599243
Epoch	2 Iter	99 Loss	0.500091552734375
Epoch	2 Iter	199 Loss	0.5027673840522766
Epoch	2 Iter	299 Loss	0.5058207511901855
Epoch	2 Iter	399 Loss	0.500218391418457
Epoch	2 Iter	499 Loss	0.5092052221298218
Epoch	2 Iter	599 Loss	0.5079705119132996
Epoch	2 Iter	699 Loss	

KeyboardInterrupt: ignored

In [0]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
                        nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=1, bias=True),
                        nn.BatchNorm2d(num_features=16, track_running_stats=True),
                        nn.ReLU(),
                        nn.Conv2d(16, 32, kernel_size=5, stride=2, padding=1, bias=True),
                        nn.BatchNorm2d(num_features=32, track_running_stats=True),
                        nn.ReLU(),
                        nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=1, bias=True),
                        nn.BatchNorm2d(num_features=64, track_running_stats=True),
                        nn.ReLU())
        
        self.mean = nn.Linear(256,16,bias=True)
        self.var = nn.Linear(256,16,bias=True)
        self.s = nn.Linear(256,16,bias=True)
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        z_mean = self.mean(x)
        z_var = self.var(x)
        s_ = self.s(x)
        return z_mean, z_var, s_

In [0]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.z_input = nn.Linear(16, 256, bias=True)
        self.s_input = nn.Linear(16, 256, bias=True)
        self.transposeConv = nn.Sequential(
                                nn.ConvTranspose2d(128, 32, kernel_size=4, stride=2, padding=0, bias=True),
                                nn.BatchNorm2d(32),
                                nn.LeakyReLU(negative_slope=0.2),
                                nn.ConvTranspose2d(32, 16, kernel_size=4, stride=2, padding=0, bias=True),
                                nn.BatchNorm2d(16),
                                nn.LeakyReLU(negative_slope=0.2),
                                nn.ConvTranspose2d(16, 1, kernel_size=4, stride=2, padding=1, bias=True),
                                nn.Sigmoid())
    
    def forward(self, z_embeddings, s_embeddings):
        z_embeddings = F.leaky_relu_(self.z_input(z_embeddings), negative_slope=0.2)
        s_embeddings = F.leaky_relu_(self.s_input(s_embeddings), negative_slope=0.2)

        x = torch.cat((z_embeddings, s_embeddings), dim=1)
        x = x.view(x.size(0), 128, 2, 2)
        x = self.transposeConv(x)
        return x

In [0]:
def reparameterize(training, mean, var):
    if training:
        std = var.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size()).normal_())
        return eps.mul(std).add_(mean)
    else:
        return mean

In [0]:
def mse_loss(x_1, x_2):
    return torch.sum((x_1-x_2).pow(2))/x_1.data.nelement()

In [0]:
def imshow_grid(images, shape=[8, 8], name='default', save=False):
    fig = plt.figure(1)
    grid = ImageGrid(fig, 111, nrows_ncols=shape, axes_pad=0.05)
    
    size = shape[0] * shape[1]
    for i in range(size):
        grid[i].axis('off')
        grid[i].imshow(images[i])

    if save:
        plt.savefig(path + str(name) + '.png')
        plt.clf()
    else:
        plt.show()

In [0]:
class MNIST_Paired():
    def __init__(self):
        self.mnist = torchvision.datasets.MNIST(root=path, download=True, train=True, transform=torchvision.transforms.Compose([
                                                     torchvision.transforms.ToTensor(),
                                                     torchvision.transforms.Normalize((0.1307,), (0.3081,))
                                                     ]))
                                                                                                          
        self.data_dict = {}

        for i in range(self.__len__()):
            image, label = self.mnist.__getitem__(i)

            try:
                self.data_dict[label]
            except KeyError:
                self.data_dict[label] = []
            self.data_dict[label].append(image)

    def __len__(self):
        return self.mnist.__len__()

    def __getitem__(self, index):
        image, label = self.mnist.__getitem__(index)
        return image, random.SystemRandom().choice(self.data_dict[label]), label

In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = [
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        ]
        for idx, module in enumerate(self.main):
            self.add_module(str(idx), module)
    def forward(self, x):
        for layer in self.main:
            x = layer(x)
        return x

In [0]:
def weights_init(layer):
    if isinstance(layer, nn.Conv2d):
        layer.weight.data.normal_(0.0, 0.05)
        layer.bias.data.zero_()
    elif isinstance(layer, nn.BatchNorm2d):
        layer.weight.data.normal_(1.0, 0.02)
        layer.bias.data.zero_()
    elif isinstance(layer, nn.Linear):
        layer.weight.data.normal_(0.0, 0.05)
        layer.bias.data.zero_()


In [65]:
plt.plot(range(len(loss_plot), loss_plot))
plt.show()
         

TypeError: ignored